In [2]:
import socket
import threading
import datetime
import logging
import json
import os
import random
import pickle

client_sockets = set()

class ThreadedServer():
    def __init__(self, host = '', port = 9091):
        self.host = host
        self.port = port
        self.socket = socket.socket()
        self.socket.bind((self.host, self.port))
        Log_Format = "%(levelname)s %(asctime)s: %(message)s"
        logging.basicConfig(filename = "logfile.log", filemode = "w", format = Log_Format, level = logging.DEBUG)
        self.logger = logging.getLogger()
        
    def listen(self, num = 10):
        self.socket.listen(num)
        while True:
            client_conn, client_addr = self.socket.accept()
            self.logger.info(f'Создано подключение: {client_addr}')
            threading.Thread(target = self.client_processing, args = (client_conn, client_addr)).start()
            
    def client_processing(self, conn, addr, data_batch = 1024):
        address = conn.recv(1024).decode('utf-8')
        if os.stat('identification.json').st_size == 0:
            names = []
            new = {}
            conn.send('None'.encode('utf-8'))
            name = conn.recv(1024).decode('utf-8')
            new['address'] = address
            new['name'] = name
            names.append(new)
            with open('identification.json', 'w', encoding='utf-8') as f:
                json.dump(names, f)
        else:
            with open('identification.json', 'r', encoding='utf-8') as f:
                names = json.load(f)
                a = 0
                for el in names:
                    if el['address'] == address:
                        name = el['name']
                        conn.send(name.encode('utf-8'))
                        a = 1
            if a == 0:
                with open('identification.json', 'r', encoding='utf-8') as f:
                    names = json.load(f)
                new = {}
                conn.send('None'.encode('utf-8'))
                name = conn.recv(1024).decode('utf-8')
                new['address'] = address
                new['name'] = name
                names.append(new)
                with open('identification.json', 'w', encoding='utf-8') as f:
                    json.dump(names, f, indent=1)
                    
        num = conn.recv(1024).decode('utf-8') #получение ключа 
        new = self.key(num)
        conn.send(str(new[0]).encode('utf-8'))
        global K
        K = new[1]
        
        while True:
            data = conn.recv(data_batch)
            if not data:
                self.logger.info(f'Соединение с {addr} разорвано')
                break
            data_decode = self.fr_cipher(data)
            if data_decode == 'commands':
                co = conn.recv(data_batch)
                if not co:
                    self.logger.info(f'Соединение с {addr} разорвано')
                    break
                else:
                    com = self.fr_cipher(co)
                    if com == '0':
                        self.logger.info('Отключение сервера')
                        for cs in client_sockets:
                            cs.close()
                        self.socket.close()
                    else:
                        mes = self.commands(com)
                        if com == '2':
                            conn.send(mes.encode('utf-8'))
                        else:
                            conn.send(self.to_cipher(mes))
            else:
                self.logger.info(f'Получены данные от клиента: {data_decode}')
                self.logger.info(f'Отправлены данные клиенту: {data_decode.upper()}')
                conn.send(self.to_cipher(data_decode.upper()))
        self.logger.info(f'Закрываем соединение с {addr}')
        conn.close()
        
    def key(self, num):
        numb=num.split()
        A=int(numb[0])
        g=int(numb[1])
        p=int(numb[2])
        b=random.randint(50, 100)
        B=(g**b)%p
        K=(A**b)%p
        new = [B, K]
        return new
    
    def to_cipher(self, mes): # шифрование
        mess = []
        for symbol in mes:
            mess.append(chr(ord(symbol)+K))
        return pickle.dumps(mess)

    def fr_cipher(self, mes):
        mes_n = pickle.loads(mes)
        mess = []
        for symbol in mes_n:
            mess.append(chr(ord(symbol)-K))
        mess_n = ''.join(mess)
        return mess_n

    def commands(self, com):
        command_dict = {2: 'self.show', 3: 'self.clean_logs', 4: 'self.clean_ident'}
        if int(com) in [2,3,4]:
            func = command_dict[int(com)]
            return eval(f'{func}()')
        else:
            return "Error!"
        
    def show(self):
        with open("logfile.log", "r") as file:
            logs = file.read()
        if logs == '':
            return 'No data'
        else:
            return logs
                                
    def clean_logs(self):
        with open("logfile.log", "w"):
            pass
        return 'Файл очищен'
    
    def clean_ident(self):
        with open("identification.json", "w") as file:
            pass
        return 'Файл очищен'
        
    
        
server = ThreadedServer()

In [ ]:
server.listen(5)